# 08. Failure Analysis and Guardrails

Map failure patterns to concrete guardrails.


In [ ]:
from __future__ import annotations

import json
import os
import math
import random
import statistics
from pathlib import Path


def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'README.md').exists() and (candidate / 'main_langgraph.py').exists():
            return candidate
    return start


PROJECT_ROOT = find_project_root(Path.cwd().resolve())
os.chdir(PROJECT_ROOT)
print('PROJECT_ROOT =', PROJECT_ROOT)


In [ ]:
synthetic_errors = [
    'ERROR: Invalid calculator expression format. Expected arithmetic expression, got: Calculate CAGR from step 3',
    'ERROR: Unsupported tool unknown_tool',
    'ERROR: All search providers failed for query: market size',
    'plan_json_parse_failed: Expecting value',
]


def classify_error(msg: str) -> str:
    m = msg.lower()
    if 'calculator expression' in m:
        return 'input_validation'
    if 'unsupported tool' in m:
        return 'tool_contract'
    if 'search providers failed' in m:
        return 'retrieval_availability'
    if 'plan_json_parse_failed' in m:
        return 'planner_format'
    return 'other'


classes = [classify_error(e) for e in synthetic_errors]
counts = {c: classes.count(c) for c in sorted(set(classes))}
print(counts)


In [ ]:
guardrails = {
    'input_validation': 'Validate calculator/tool inputs before execution',
    'tool_contract': 'Whitelist tools and schema-check each step',
    'retrieval_availability': 'Add retries and multi-provider fallback',
    'planner_format': 'Use strict JSON schema + fallback plan',
}

for k, v in guardrails.items():
    print(f'{k}: {v}')

assert 'input_validation' in counts
assert 'planner_format' in counts
print('Failure taxonomy and guardrails are defined.')
